In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from fbprophet import Prophet
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

In [11]:
#DATA PREPARATION FOR UNIVARIATE PROPHET MODEL

#GATHER Paris DATA FROM ALL AVAILABLE YEARS
Paris_Data = pd.read_csv('Paris_data.csv')
Paris_Data.rename(columns = {Paris_Data.columns[0]:'ds'}, inplace = True)

#KEEP ONLY THE POLLUTANTS COLUMNS
cols=['pm25', 'pm10', 'no2', 'o3', 'so2', 'ds']

#MAKE VALUES NUMERIC
Paris_Data_uni = Paris_Data.loc[:, Paris_Data.columns.intersection(cols)]
Paris_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']] = Paris_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']].apply(pd.to_numeric, errors='coerce')

#FILL NAN VALUES
Paris_Data_uni = Paris_Data_uni.iloc[1: , :]

#SOME VALUES ARE EMPTY, BUT NOT RECOGNIZED AS NAN. REPLACE EMPTY VALUES WITH NAN TO FILL THEM LATER
Paris_Data_uni = Paris_Data_uni.replace(r'^\s*$', np.nan, regex=True)
Paris_Data_uni = Paris_Data_uni.fillna(method='ffill')
Paris_Data_uni = Paris_Data_uni.fillna(method='bfill')

#REMOVE DUPLICATE VALUES
Paris_Data_uni = Paris_Data_uni[~Paris_Data_uni.index.duplicated(keep='first')]

#Fb prophet model predicts univariate time series. It requires that the Date index column is named 'ds'
#and the values column is named 'y'
Paris_Data_uni['ds'] = pd.DatetimeIndex(Paris_Data_uni['ds'])

In [3]:
Paris_Data_uni

,ds,no2,o3,pm10,pm25,so2
1,2014-12-30,0.567033,0.086777,0.338235,0.372549,0.204082
2,2014-12-31,0.778022,0.070248,0.544118,0.542484,0.204082
3,2015-01-01,0.679121,0.188017,0.529412,0.666667,0.153061
4,2015-01-02,0.567033,0.128099,0.308824,0.405229,0.102041
5,2015-01-03,0.356044,0.163223,0.102941,0.228758,0.051020
...,...,...,...,...,...,...
1869,2022-03-14,0.285714,0.450413,0.161765,0.156863,0.020408
1870,2022-03-15,0.463736,0.198347,0.147059,0.235294,0.020408
1871,2022-03-16,0.373626,0.126033,0.397059,0.326797,0.020408
1872,2022-03-17,0.178022,0.469008,0.147059,0.183007,0.020408


In [4]:
#Check data for stationarity to apply the ARIMA model.

#The null hypothesis is that the time series is non-stationary.
#Using the Augmented Dickey Fuller test (adfuller()) we can check for stationarity.
#If p-value < significance level(0.05), the we reject the null hypothesis.
#Otherwise, the time series is non-stationary and needs differencing.

print("Check if Paris_Data is stationary :")
for col in ['no2', 'o3', 'so2', 'pm10', 'pm25']:
    result = adfuller(Paris_Data[col])
    print('p-value for ' + col + ' : %f' % result[1])

Check if Paris_Data is stationary :
p-value for no2 : 0.000088
p-value for o3 : 0.000176
p-value for so2 : 0.000694
p-value for pm10 : 0.000000
p-value for pm25 : 0.000000


In [6]:
def fbprophet_predict(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    
    #SPLIT TO TRAIN AND TEST PORTIONS
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Paris[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Paris[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Paris[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [6]:
#FB PROPHET REQUIRES THE DATAFRAME TO HAVE TWO COLUMNS. 'ds' AND 'y', 'ds' IS THE TIME INDEX AND 'y' IS THE
#VALUE OF THE POLLUTANT

#RENAME COLUMNS TO FIT TO THE PROPHET MODEL
Paris_o3 = Paris_Data_uni[['ds', 'o3']]
Paris_o3.rename(columns = {'o3':'y'}, inplace = True)

Paris_no2 = Paris_Data_uni[['ds', 'no2']]
Paris_no2.rename(columns = {'no2':'y'}, inplace = True)

Paris_so2 = Paris_Data_uni[['ds', 'so2']]
Paris_so2.rename(columns = {'so2':'y'}, inplace = True)

Paris_pm10 = Paris_Data_uni[['ds', 'pm10']]
Paris_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Paris_pm25 = Paris_Data_uni[['ds', 'pm25']]
Paris_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Paris_o3, Paris_no2, Paris_so2, Paris_pm10, Paris_pm25]

In [7]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.304903    0.146571    0.476278
1 2021-02-12  0.302925    0.148497    0.469827
2 2021-02-13  0.324316    0.155893    0.490084
3 2021-02-14  0.349136    0.181786    0.521421
4 2021-02-15  0.328419    0.166795    0.490660
          ds         y
0 2021-02-11  0.334711
1 2021-02-12  0.390496
2 2021-02-13  0.413223
3 2021-02-14  0.448347
4 2021-02-15  0.210744
FB Prophet RMSE for Paris[o3]: 0.14040385530622787

FB Prophet MSE for Paris[o3]: 0.019713242584852173

FB Prophet MAE for Paris[o3]: 0.11053011715284723



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.281792    0.114005    0.468590
1 2021-02-12  0.297582    0.120256    0.477113
2 2021-02-13  0.225971    0.055031    0.406273
3 2021-02-14  0.165142   -0.000622    0.346188
4 2021-02-15  0.231551    0.058717    0.406649
          ds         y
0 2021-02-11  0.283516
1 2021-02-12  0.215385
2 2021-02-13  0.261538
3 2021-02-14  0.226374
4 2021-02-15  0.367033
FB Prophet RMSE for Paris[no2]: 0.15661167644307863

FB Prophet MSE for Paris[no2]: 0.024527217198311548

FB Prophet MAE for Paris[no2]: 0.11484689816159183



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.085145    0.030368    0.140751
1 2021-02-12  0.082198    0.025229    0.136228
2 2021-02-13  0.076322    0.022860    0.128092
3 2021-02-14  0.072999    0.016752    0.128812
4 2021-02-15  0.080989    0.019371    0.138714
          ds         y
0 2021-02-11  0.051020
1 2021-02-12  0.153061
2 2021-02-13  0.071429
3 2021-02-14  0.173469
4 2021-02-15  0.091837
FB Prophet RMSE for Paris[so2]: 0.06615917148026548

FB Prophet MSE for Paris[so2]: 0.004377035970955174

FB Prophet MAE for Paris[so2]: 0.05587873154289234



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.253615    0.073846    0.423745
1 2021-02-12  0.270857    0.107568    0.448472
2 2021-02-13  0.227219    0.044179    0.404930
3 2021-02-14  0.196381    0.021692    0.366414
4 2021-02-15  0.222861    0.046887    0.408729
          ds         y
0 2021-02-11  0.264706
1 2021-02-12  0.205882
2 2021-02-13  0.205882
3 2021-02-14  0.220588
4 2021-02-15  0.294118
FB Prophet RMSE for Paris[pm10]: 0.13933367835111757

FB Prophet MSE for Paris[pm10]: 0.019413873922852688

FB Prophet MAE for Paris[pm10]: 0.10222987534025897

          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.281457    0.093659    0.457265
1 2021-02-12  0.293338    0.117114    0.486780
2 2021-02-13  0.282900    0.097348    0.466346
3 2021-02-14  0.259985    0.075051    0.440855
4 2021-02-15  0.254184    0.071365    0.437703
          ds         y
0 2021-02-11  0.359477
1 2021-02-12  0.274510
2 2021-02-13  0.274510
3 2021-02-14  0.313725
4 2021-02-15  0.313725

### Repeat process but this time train model with the same amount of data as Athens, to prove that the model returns better results due to bigger amount of data.

In [8]:
import pandas as pd
Athens_Data = pd.read_csv('Athens_Data.csv', index_col=0)

In [9]:
len(Athens_Data)

839

In [10]:
data_to_drop = len(Paris_Data) - len(Athens_Data)

In [11]:
Paris_less_data = Paris_Data.iloc[data_to_drop: , :]
len(Paris_less_data)

839

In [12]:
Paris_less_data

,ds,no2,o3,pm10,pm25,so2,temp,dew,humidity,windspeed,winddir
1035,2019-11-05,0.397802,0.291322,0.176471,0.176471,0.091837,0.360000,0.614035,0.770794,0.251908,0.604913
1036,2019-11-06,0.349451,0.171488,0.176471,0.254902,0.040816,0.376000,0.640351,0.799524,0.170483,0.733057
1037,2019-11-07,0.259341,0.295455,0.102941,0.124183,0.030612,0.344000,0.578947,0.726984,0.455471,0.595146
1038,2019-11-08,0.479121,0.107438,0.205882,0.228758,0.010204,0.277333,0.532164,0.786825,0.129771,0.413732
1039,2019-11-09,0.364835,0.165289,0.235294,0.300654,0.020408,0.266667,0.529240,0.817778,0.302799,0.632436
...,...,...,...,...,...,...,...,...,...,...,...
1869,2022-03-14,0.285714,0.450413,0.161765,0.156863,0.020408,0.357333,0.540936,0.603175,0.206107,0.444510
1870,2022-03-15,0.463736,0.198347,0.147059,0.235294,0.020408,0.373333,0.652047,0.847619,0.170483,0.311927
1871,2022-03-16,0.373626,0.126033,0.397059,0.326797,0.020408,0.402667,0.666667,0.793651,0.376590,0.438591
1872,2022-03-17,0.178022,0.469008,0.147059,0.183007,0.020408,0.338667,0.540936,0.614286,0.402036,0.371412


In [13]:
Paris_o3 = Paris_less_data[['ds', 'o3']]
Paris_o3.rename(columns = {'o3':'y'}, inplace = True)

Paris_no2 = Paris_less_data[['ds', 'no2']]
Paris_no2.rename(columns = {'no2':'y'}, inplace = True)

Paris_so2 = Paris_less_data[['ds', 'so2']]
Paris_so2.rename(columns = {'so2':'y'}, inplace = True)

Paris_pm10 = Paris_less_data[['ds', 'pm10']]
Paris_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Paris_pm25 = Paris_less_data[['ds', 'pm25']]
Paris_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Paris_o3, Paris_no2, Paris_so2, Paris_pm10, Paris_pm25]

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.566563    0.375406    0.735485
1 2021-09-07  0.568175    0.408108    0.751513
2 2021-09-08  0.585721    0.397458    0.780580
3 2021-09-09  0.583086    0.398909    0.757503
4 2021-09-10  0.583267    0.397971    0.768493
           ds         y
0  2021-09-06  0.617769
1  2021-09-07  0.595041
2  2021-09-08  0.719008
3  2021-09-09  0.489669
4  2021-09-10  0.433884
FB Prophet RMSE for Paris[o3]: 0.4361973760740367

FB Prophet MSE for Paris[o3]: 0.19026815089387458

FB Prophet MAE for Paris[o3]: 0.39683500854401044



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.143381   -0.005000    0.296172
1 2021-09-07  0.180431    0.019570    0.340214
2 2021-09-08  0.175753    0.021143    0.334819
3 2021-09-09  0.176274    0.023176    0.328618
4 2021-09-10  0.165014    0.012091    0.315173
           ds         y
0  2021-09-06  0.470330
1  2021-09-07  0.404396
2  2021-09-08  0.283516
3  2021-09-09  0.232967
4  2021-09-10  0.186813
FB Prophet RMSE for Paris[no2]: 0.24653623502169442

FB Prophet MSE for Paris[no2]: 0.06078011517867215

FB Prophet MAE for Paris[no2]: 0.21234174186508983



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.040428   -0.021738    0.110076
1 2021-09-07  0.040146   -0.026781    0.109824
2 2021-09-08  0.033680   -0.030401    0.102186
3 2021-09-09  0.046439   -0.021474    0.109423
4 2021-09-10  0.037743   -0.030273    0.105233
           ds         y
0  2021-09-06  0.061224
1  2021-09-07  0.020408
2  2021-09-08  0.020408
3  2021-09-09  0.020408
4  2021-09-10  0.020408
FB Prophet RMSE for Paris[so2]: 0.057367997211116524

FB Prophet MSE for Paris[so2]: 0.0032910871040146734

FB Prophet MAE for Paris[so2]: 0.02509709309654034



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.187531    0.036703    0.342162
1 2021-09-07  0.204419    0.037965    0.367937
2 2021-09-08  0.221535    0.061399    0.372842
3 2021-09-09  0.203323    0.048359    0.357098
4 2021-09-10  0.199551    0.037931    0.352610
           ds         y
0  2021-09-06  0.279412
1  2021-09-07  0.294118
2  2021-09-08  0.308824
3  2021-09-09  0.220588
4  2021-09-10  0.117647
FB Prophet RMSE for Paris[pm10]: 0.14458849124806047

FB Prophet MSE for Paris[pm10]: 0.02090583180139046

FB Prophet MAE for Paris[pm10]: 0.10604562368566056

          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.166257    0.009858    0.322307
1 2021-09-07  0.180823    0.020517    0.337962
2 2021-09-08  0.194913    0.054607    0.347879
3 2021-09-09  0.175504    0.015436    0.334302
4 2021-09-10  0.177639    0.026039    0.336852
           ds         y
0  2021-09-06  0.281046
1  2021-09-07  0.281046
2  2021-09-08  0.313725
3  2021-09-09  0.281046
4  2021-09-1

## Seasonal split

The seasonal decomposition of the Athens data we can see that there is a seasonality in the monthly resampled data. Specifically, for each pollutant the seasonal length of the data is:
- NO2: 12 months or 365 days
- SO2: 12 months
- O3: 12 months
- PM25: 12 months
- PM10: 12 months

In [16]:
def fbprophet_predict_seasonal(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    time_series.index = time_series['ds']
    #SPLIT TO TRAIN AND TEST PORTIONS
    train, test = time_series['2019-12-01' : '2021-12-31'], time_series['2022-01-01' : '2022-03-18']
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Paris[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Paris[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Paris[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [17]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict_seasonal(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.222058    0.052881    0.392880
1 2022-01-02  0.214034    0.052761    0.367417
2 2022-01-03  0.193336    0.034234    0.352133
3 2022-01-04  0.187424    0.028188    0.343688
4 2022-01-05  0.199751    0.046068    0.349868
           ds         y
0  2022-01-01  0.033058
1  2022-01-02  0.276860
2  2022-01-03  0.475207
3  2022-01-04  0.438017
4  2022-01-05  0.357438
FB Prophet RMSE for Paris[o3]: 0.1497421780245382

FB Prophet MSE for Paris[o3]: 0.022422719879532496

FB Prophet MAE for Paris[o3]: 0.1266144652434164



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.252954    0.111725    0.402650
1 2022-01-02  0.205444    0.046422    0.357975
2 2022-01-03  0.284564    0.126455    0.426771
3 2022-01-04  0.329864    0.183207    0.492896
4 2022-01-05  0.330182    0.187423    0.480070
           ds         y
0  2022-01-01  0.261538
1  2022-01-02  0.145055
2  2022-01-03  0.153846
3  2022-01-04  0.184615
4  2022-01-05  0.265934
FB Prophet RMSE for Paris[no2]: 0.12738623386716266

FB Prophet MSE for Paris[no2]: 0.01622725257885946

FB Prophet MAE for Paris[no2]: 0.10344164149115218



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.042705   -0.026407    0.112813
1 2022-01-02  0.045044   -0.023190    0.112103
2 2022-01-03  0.048299   -0.019418    0.116362
3 2022-01-04  0.048778   -0.020188    0.114106
4 2022-01-05  0.045093   -0.026235    0.107747
           ds         y
0  2022-01-01  0.010204
1  2022-01-02  0.020408
2  2022-01-03  0.010204
3  2022-01-04  0.010204
4  2022-01-05  0.010204
FB Prophet RMSE for Paris[so2]: 0.03243103395789798

FB Prophet MSE for Paris[so2]: 0.001051771963578332

FB Prophet MAE for Paris[so2]: 0.018605271029542982



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.170850    0.021929    0.321832
1 2022-01-02  0.136596   -0.015743    0.280581
2 2022-01-03  0.185804    0.035502    0.323902
3 2022-01-04  0.216058    0.059081    0.369529
4 2022-01-05  0.234124    0.071494    0.398938
           ds         y
0  2022-01-01  0.205882
1  2022-01-02  0.132353
2  2022-01-03  0.176471
3  2022-01-04  0.014706
4  2022-01-05  0.132353
FB Prophet RMSE for Paris[pm10]: 0.16307555360221768

FB Prophet MSE for Paris[pm10]: 0.026593636182669772

FB Prophet MAE for Paris[pm10]: 0.1188243512464408

          ds      yhat  yhat_lower  yhat_upper
0 2022-01-01  0.310182    0.160586    0.456511
1 2022-01-02  0.286593    0.137914    0.430675
2 2022-01-03  0.300643    0.148640    0.452415
3 2022-01-04  0.326073    0.181378    0.466971
4 2022-01-05  0.343635    0.197509    0.486342
           ds         y
0  2022-01-01  0.300654
1  2022-01-02  0.189542
2  2022-01-03  0.169935
3  2022-01-04  0.039216
4  2022-01-0

## Weekly predictions

In [3]:
Paris_Data_uni.index = Paris_Data_uni['ds']
Paris_Data_uni = Paris_Data_uni.drop(['ds'],axis=1)

Paris_Data_uni.index = pd.to_datetime(Paris_Data_uni.index)
Paris_weekly = Paris_Data_uni.resample("1w").mean().bfill()

Paris_weekly['ds'] = Paris_weekly.index

In [4]:
Paris_o3 = Paris_weekly[['ds', 'o3']]
Paris_o3.rename(columns = {'o3':'y'}, inplace = True)

Paris_no2 = Paris_weekly[['ds', 'no2']]
Paris_no2.rename(columns = {'no2':'y'}, inplace = True)

Paris_so2 = Paris_weekly[['ds', 'so2']]
Paris_so2.rename(columns = {'so2':'y'}, inplace = True)

Paris_pm10 = Paris_weekly[['ds', 'pm10']]
Paris_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Paris_pm25 = Paris_weekly[['ds', 'pm25']]
Paris_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Paris_o3, Paris_no2, Paris_so2, Paris_pm10, Paris_pm25]

In [10]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.372101    0.258302    0.495956
1 2020-10-18  0.367534    0.262444    0.476860
2 2020-10-25  0.354349    0.236140    0.463289
3 2020-11-01  0.341751    0.231486    0.451066
4 2020-11-08  0.339119    0.225082    0.447845
          ds         y
0 2020-10-11  0.384593
1 2020-10-18  0.183884
2 2020-10-25  0.323790
3 2020-11-01  0.356848
4 2020-11-08  0.216942
FB Prophet RMSE for Paris[o3]: 0.14517097104395482

FB Prophet MSE for Paris[o3]: 0.021074610833844768

FB Prophet MAE for Paris[o3]: 0.11361127692334627



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.227992    0.115376    0.341283
1 2020-10-18  0.232832    0.122118    0.348502
2 2020-10-25  0.238828    0.122018    0.350044
3 2020-11-01  0.242487    0.129142    0.352928
4 2020-11-08  0.244883    0.136910    0.353873
          ds         y
0 2020-10-11  0.131240
1 2020-10-18  0.262794
2 2020-10-25  0.233281
3 2020-11-01  0.144741
4 2020-11-08  0.264364
FB Prophet RMSE for Paris[no2]: 0.08718390818992292

FB Prophet MSE for Paris[no2]: 0.007601033847268907

FB Prophet MAE for Paris[no2]: 0.06581686386978292



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.064459    0.034155    0.097561
1 2020-10-18  0.058194    0.028657    0.087666
2 2020-10-25  0.052767    0.024106    0.083089
3 2020-11-01  0.052007    0.021011    0.084359
4 2020-11-08  0.055498    0.023792    0.086341
          ds         y
0 2020-10-11  0.052478
1 2020-10-18  0.104956
2 2020-10-25  0.052478
3 2020-11-01  0.034985
4 2020-11-08  0.062682
FB Prophet RMSE for Paris[so2]: 0.02338651063098514

FB Prophet MSE for Paris[so2]: 0.0005469288794931809

FB Prophet MAE for Paris[so2]: 0.019143010743818984



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.164561    0.049473    0.287243
1 2020-10-18  0.174715    0.050613    0.300744
2 2020-10-25  0.189237    0.067427    0.314751
3 2020-11-01  0.190509    0.072313    0.323048
4 2020-11-08  0.178568    0.049809    0.297819
          ds         y
0 2020-10-11  0.098739
1 2020-10-18  0.197479
2 2020-10-25  0.174370
3 2020-11-01  0.113445
4 2020-11-08  0.222689
FB Prophet RMSE for Paris[pm10]: 0.09120885869905562

FB Prophet MSE for Paris[pm10]: 0.008319055905184294

FB Prophet MAE for Paris[pm10]: 0.06956344989020022

          ds      yhat  yhat_lower  yhat_upper
0 2020-10-11  0.187308    0.062804    0.312847
1 2020-10-18  0.196703    0.070429    0.324322
2 2020-10-25  0.211068    0.079310    0.337462
3 2020-11-01  0.216760    0.082887    0.347274
4 2020-11-08  0.211830    0.081570    0.342599
          ds         y
0 2020-10-11  0.094304
1 2020-10-18  0.243697
2 2020-10-25  0.169935
3 2020-11-01  0.112979
4 2020-11-08  0.237162

## Monthly predictions

In [12]:
Paris_Data_uni.index = Paris_Data_uni['ds']
Paris_Data_uni = Paris_Data_uni.drop(['ds'],axis=1)

Paris_Data_uni.index = pd.to_datetime(Paris_Data_uni.index)
Paris_monthly = Paris_Data_uni.resample("1m").mean().bfill()

Paris_monthly['ds'] = Paris_monthly.index

In [13]:
Paris_o3 = Paris_monthly[['ds', 'o3']]
Paris_o3.rename(columns = {'o3':'y'}, inplace = True)

Paris_no2 = Paris_monthly[['ds', 'no2']]
Paris_no2.rename(columns = {'no2':'y'}, inplace = True)

Paris_so2 = Paris_monthly[['ds', 'so2']]
Paris_so2.rename(columns = {'so2':'y'}, inplace = True)

Paris_pm10 = Paris_monthly[['ds', 'pm10']]
Paris_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Paris_pm25 = Paris_monthly[['ds', 'pm25']]
Paris_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Paris_o3, Paris_no2, Paris_so2, Paris_pm10, Paris_pm25]

In [14]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    
    i = i+1

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.290226    0.195257    0.385050
1 2020-11-30  0.274461    0.172869    0.370056
2 2020-12-31  0.253534    0.161003    0.349339
3 2021-01-31  0.258354    0.164440    0.358109
4 2021-02-28  0.394618    0.294359    0.497802
          ds         y
0 2020-10-31  0.311917
1 2020-11-30  0.194284
2 2020-12-31  0.242069
3 2021-01-31  0.229539
4 2021-02-28  0.312721
FB Prophet RMSE for Paris[o3]: 0.09082287819249651

FB Prophet MSE for Paris[o3]: 0.008248795203169058

FB Prophet MAE for Paris[o3]: 0.07670269081045662



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.284694    0.210461    0.368099
1 2020-11-30  0.285239    0.205170    0.371040
2 2020-12-31  0.320535    0.243109    0.402271
3 2021-01-31  0.321697    0.241517    0.406784
4 2021-02-28  0.254568    0.176030    0.333577
          ds         y
0 2020-10-31  0.200638
1 2020-11-30  0.293626
2 2020-12-31  0.247714
3 2021-01-31  0.273591
4 2021-02-28  0.284694
FB Prophet RMSE for Paris[no2]: 0.07301075807144099

FB Prophet MSE for Paris[no2]: 0.005330570794166486

FB Prophet MAE for Paris[no2]: 0.059879918846896016



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.043747    0.017573    0.067179
1 2020-11-30  0.049214    0.025891    0.074254
2 2020-12-31  0.068153    0.043817    0.093162
3 2021-01-31  0.082523    0.056570    0.107095
4 2021-02-28  0.053604    0.029018    0.077906
          ds         y
0 2020-10-31  0.092495
1 2020-11-30  0.055102
2 2020-12-31  0.055958
3 2021-01-31  0.068466
4 2021-02-28  0.058309
FB Prophet RMSE for Paris[so2]: 0.019833598221029994

FB Prophet MSE for Paris[so2]: 0.0003933716183932441

FB Prophet MAE for Paris[so2]: 0.016070922969550216



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.141963    0.054080    0.237173
1 2020-11-30  0.154704    0.062510    0.245451
2 2020-12-31  0.198735    0.105131    0.288477
3 2021-01-31  0.224300    0.136270    0.320581
4 2021-02-28  0.189578    0.098293    0.283347
          ds         y
0 2020-10-31  0.143738
1 2020-11-30  0.266667
2 2020-12-31  0.139469
3 2021-01-31  0.206357
4 2021-02-28  0.266807
FB Prophet RMSE for Paris[pm10]: 0.07113926346765148

FB Prophet MSE for Paris[pm10]: 0.0050607948067199334

FB Prophet MAE for Paris[pm10]: 0.059217492950274994

          ds      yhat  yhat_lower  yhat_upper
0 2020-10-31  0.193550    0.113100    0.273782
1 2020-11-30  0.197012    0.110481    0.274295
2 2020-12-31  0.230831    0.150765    0.309696
3 2021-01-31  0.278330    0.193387    0.368252
4 2021-02-28  0.207962    0.127624    0.290174
          ds         y
0 2020-10-31  0.153279
1 2020-11-30  0.281481
2 2020-12-31  0.180266
3 2021-01-31  0.296226
4 2021-02-28  0.2796